In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm

import sys
sys.path.append('..')

import modules.mean_shift as ms
import modules.general as gen
import modules.plotting as pl
import modules.pandas_funcs as pf
import modules.pose_estimation as pe
import modules.math_funcs as mf
import modules.signals as sig
import modules.gait_metrics as gm
import modules.linear_algebra as lin
import modules.point_processing as pp

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
directory = os.path.join('..', 'data', 'pickles')

file_name = '2014-12-16_P005_Pre_002_df_pass.pkl'
file_path = os.path.join(directory, file_name)

df_pass = pd.read_pickle(file_path)
frames = df_pass.index.values

In [4]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

## Cluster foot distance 

In [5]:
def make_consistent(df_walk, direction):
                
    # Establish a motion correspondence so the foot sides do not switch abruptly
    foot_points_l, foot_points_r = pp.track_two_objects(df_walk.L_FOOT, df_walk.R_FOOT)
    
    df_assigned = df_walk.copy()
    df_assigned.L_FOOT = pf.series_of_rows(foot_points_l, index=df_walk.index)
    df_assigned.R_FOOT = pf.series_of_rows(foot_points_r, index=df_walk.index)
    
    head_points = np.stack(df_walk.HEAD)
    side_values = pe.evaluate_foot_side(head_points, foot_points_l, foot_points_r, direction)
    
    if sum(side_values) > 0:
        
        # The left foot should be labelled the right foot, and vice versa
        df_assigned = pf.swap_columns(df_assigned, 'L_FOOT', 'R_FOOT')
    
    return df_assigned

In [6]:
def enforce_consistency(df_pass):
    
    head_points = np.stack(df_pass.HEAD)
    _, direction_pass = lin.best_fit_line(head_points)
    
    foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)
    
    deriv = sig.window_derivative(foot_dist, 5)
    deriv_2 = sig.window_derivative(deriv, 5).dropna()

    masses = mf.normalize_array(deriv_2.values)
    peak_frames, mid_frames = sig.mean_shift_peaks(deriv_2, masses=masses, kernel='gaussian', radius=3)
    
    labels = gen.label_by_split(frames, peak_frames)

    grouped_dfs = list(gen.group_by_label(df_pass, labels))
    
    consistent_dfs = [make_consistent(x, direction_pass) for x in grouped_dfs]
    
    return pd.concat(consistent_dfs)

In [7]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

In [8]:
df_cons_pass = enforce_consistency(df_pass)

_, direction_pass = pe.direction_of_pass(df_cons_pass)

signal_left = gm.foot_signal(df_cons_pass.L_FOOT, df_cons_pass.R_FOOT, direction_pass)

plt.figure()
plt.plot(signal_left)
plt.show()

## View multiple frames

In [10]:
df = df_cons_pass

In [11]:
fig, ax = plt.subplots()

ax = Axes3D(fig)

for frame in range(300, 370):
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue
    
    points = np.stack(positions)
    points_to_plot = points[:, [0, 2, 1]]

    head, foot_l, foot_r = points_to_plot

    pl.scatter3(ax, head, color='k')
    pl.scatter3(ax, foot_l, color='b')
    pl.scatter3(ax, foot_r, color='r')    
    
    ax.set_xlim3d(-200, 200)
    ax.set_ylim3d(100, 300)
    ax.set_zlim3d(-100, 100)
    
    plt.title('Frame {}'.format(frame))
    
    plt.pause(0.1)

### View one frame

In [13]:
frame = 340

fig, ax = plt.subplots()

ax = Axes3D(fig)

positions = df.loc[frame]

points = np.stack(positions)
points_to_plot = points[:, [0, 2, 1]]

head, foot_l, foot_r = points_to_plot

pl.scatter3(ax, head, color='k')
pl.scatter3(ax, foot_l, color='b')
pl.scatter3(ax, foot_r, color='r')    

ax.set_xlim3d(-200, 200)
ax.set_ylim3d(100, 300)
ax.set_zlim3d(-100, 100)

plt.show()